In [10]:
eval_df.to_csv('/proj/NIRAL/users/siyangj/Bern_eval.csv',index=None)

In [10]:
import tensorflow as tf
import sys
sys.path.append('/nas/longleaf/home/siyangj/NiftyNet/')
import niftynet as nn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import SimpleITK as sitk
import os
import re
import nibabel as nib
import sklearn as skl
import pandas as pd

pd.options.display.max_colwidth = 100
pd.options.display.max_columns = 10000

In [11]:
data_path = '/proj/NIRAL/users/siyangj/myData/BernNet/Test'
inf_path  = '/proj/NIRAL/users/siyangj/myData/BernNet/Test'

In [12]:
def Dice(pred,true,labels=[0,1,2,3]):
    l = len(labels)
    ret = np.zeros(l)
    for i in range(l):
        lab = labels[i]
        p = pred==lab
        t = true==lab
        ret[i] = 2*np.logical_and(p,t).sum()/(p.sum()+t.sum())
    return ret

In [14]:
## id : T1,T2,label,inf1,inf2
def GenerateDict():
    im_range = [1,8]
    
    ret = {str(i):[
        os.path.join(data_path,'subject-{:d}'.format(i),'subject-{:d}-T1.nii.gz'.format(i)),
        os.path.join(data_path,'subject-{:d}'.format(i),'subject-{:d}-T2.nii.gz'.format(i)),
        os.path.join(data_path,'subject-{:d}'.format(i),'subject-{:d}-label.nii.gz'.format(i)),
        os.path.join(inf_path, 'subject-{:d}'.format(i),'subject-{:d}_prediction.nii.gz'.format(i)),
        os.path.join(inf_path, 'subject-{:d}'.format(i),'subject-{:d}_prediction_2stage.nii.gz'.format(i))
    ] for i in im_range}
    
    return ret

In [15]:
data_dict = GenerateDict()

In [16]:
data_dict

{'1': ['/proj/NIRAL/users/siyangj/myData/BernNet/Test/subject-1/subject-1-T1.nii.gz',
  '/proj/NIRAL/users/siyangj/myData/BernNet/Test/subject-1/subject-1-T2.nii.gz',
  '/proj/NIRAL/users/siyangj/myData/BernNet/Test/subject-1/subject-1-label.nii.gz',
  '/proj/NIRAL/users/siyangj/myData/BernNet/Test/subject-1/subject-1_prediction.nii.gz',
  '/proj/NIRAL/users/siyangj/myData/BernNet/Test/subject-1/subject-1_prediction_2stage.nii.gz'],
 '8': ['/proj/NIRAL/users/siyangj/myData/BernNet/Test/subject-8/subject-8-T1.nii.gz',
  '/proj/NIRAL/users/siyangj/myData/BernNet/Test/subject-8/subject-8-T2.nii.gz',
  '/proj/NIRAL/users/siyangj/myData/BernNet/Test/subject-8/subject-8-label.nii.gz',
  '/proj/NIRAL/users/siyangj/myData/BernNet/Test/subject-8/subject-8_prediction.nii.gz',
  '/proj/NIRAL/users/siyangj/myData/BernNet/Test/subject-8/subject-8_prediction_2stage.nii.gz']}

In [17]:
def InfDict(data_dict):
    eval_df = pd.DataFrame(columns=['subject_id',
                                '1_11','1_12','1_21','1_22',
                                '1_11+22','1_12+21',
                                '1_confusion_matrix',
                                '1_Dice0','1_Dice1','1_Dice2','1_Dice3',
                                '2_11','2_12','2_21','2_22',
                                '2_11+22','2_12+21',
                                '2_confusion_matrix',
                                '2_Dice0','2_Dice1','2_Dice2','2_Dice3'],dtype=object)
    for im_id,data_list in data_dict.items():
        
        inf1_arr = sitk.GetArrayFromImage(sitk.ReadImage(data_list[3]))
        inf2_arr = sitk.GetArrayFromImage(sitk.ReadImage(data_list[4]))
        
        true_arr = sitk.GetArrayFromImage(sitk.ReadImage(data_list[2]))
        
        Dice_1 = Dice(inf1_arr,true_arr)
        Dice_2 = Dice(inf2_arr,true_arr)
        
        c_mat1 = skl.metrics.confusion_matrix(true_arr.flatten(),inf1_arr.flatten())
        p_mat1 = c_mat1.T / c_mat1.astype(np.float).sum(axis=1)
        p_mat1 = p_mat1.T
        
        c_mat2 = skl.metrics.confusion_matrix(true_arr.flatten(),inf2_arr.flatten())
        p_mat2 = c_mat2.T / c_mat2.astype(np.float).sum(axis=1)
        p_mat2 = p_mat2.T
        
        l = len(eval_df.index)
        eval_df.loc[l,'subject_id']=im_id
        eval_df.loc[l,'1_11']=p_mat1[1,1]
        eval_df.loc[l,'1_12']=p_mat1[1,2]
        eval_df.loc[l,'1_21']=p_mat1[2,1]
        eval_df.loc[l,'1_22']=p_mat1[2,2]      
        eval_df.loc[l,'1_confusion_matrix'] = p_mat1
        
        eval_df.loc[l,'1_Dice0'] = Dice_1[0]
        eval_df.loc[l,'1_Dice1'] = Dice_1[1]
        eval_df.loc[l,'1_Dice2'] = Dice_1[2]
        eval_df.loc[l,'1_Dice3'] = Dice_1[3]
              
        eval_df.loc[l,'2_11']=p_mat2[1,1]
        eval_df.loc[l,'2_12']=p_mat2[1,2]
        eval_df.loc[l,'2_21']=p_mat2[2,1]
        eval_df.loc[l,'2_22']=p_mat2[2,2]
        eval_df.loc[l,'2_confusion_matrix'] = p_mat2
        
        eval_df.loc[l,'2_Dice0'] = Dice_2[0]
        eval_df.loc[l,'2_Dice1'] = Dice_2[1]
        eval_df.loc[l,'2_Dice2'] = Dice_2[2]
        eval_df.loc[l,'2_Dice3'] = Dice_2[3]
        
    return eval_df

In [18]:
eval_df = InfDict(data_dict)

In [19]:
eval_df

,subject_id,1_11,1_12,1_21,1_22,1_11+22,1_12+21,1_confusion_matrix,1_Dice0,1_Dice1,1_Dice2,1_Dice3,2_11,2_12,2_21,2_22,2_11+22,2_12+21,2_confusion_matrix,2_Dice0,2_Dice1,2_Dice2,2_Dice3
0,1,0.80924,0.182627,0.10623,0.85436,NaN,NaN,"[[0.9982253842811089, 0.0, 0.0009054986811654253, 0.0008691170377257431], [0.00048719723183391, ...",0.978905,0.817738,0.838279,0.747487,0.789326,0.201849,0.0869289,0.863431,NaN,NaN,"[[0.9982759143414417, 3.436044102636659e-05, 0.0003496680175036129, 0.0013400572000282968], [0.0...",0.978174,0.816113,0.840842,0.749501
1,8,0.870471,0.127562,0.0619272,0.917941,NaN,NaN,"[[0.9896514303344175, 0.00014735429959892, 0.006801321890862652, 0.003399893475120915], [2.21014...",0.992217,0.885906,0.886168,0.805581,0.8435,0.154621,0.0439909,0.934906,NaN,NaN,"[[0.9934058950929483, 0.00010130608097425751, 0.004006195020345638, 0.002486603805731775], [7.36...",0.99414,0.882703,0.892377,0.812911


In [21]:
eval_df.loc[0,'2_confusion_matrix']

array([[9.98275914e-01, 3.43604410e-05, 3.49668018e-04, 1.34005720e-03],
       [3.21107266e-04, 7.89325952e-01, 2.01849135e-01, 8.50380623e-03],
       [1.63127331e-02, 8.69288799e-02, 8.63431256e-01, 3.33271308e-02],
       [1.71181743e-01, 3.79853117e-02, 1.26014122e-01, 6.64818824e-01]])